In [2]:
import pandas as pd

df_home = pd.read_csv('./data/home_page_table.csv')
df_payment_confirm = pd.read_csv('./data/payment_confirmation_table.csv')
df_payment_page = pd.read_csv('./data/payment_page_table.csv')
df_search = pd.read_csv('./data/search_page_table.csv')
df_user_data = pd.read_csv('./data/user_table.csv')


In [61]:
from functools import reduce; 

def mergedf(df_list, on_col, how_join):
    df = reduce(lambda left,right: pd.merge(left, right, on=on_col, how=how_join), df_list)
    return df

df = mergedf([df_home, df_payment_confirm, df_payment_page, df_search, df_user_data], 'user_id', 'outer')
print(df.head(10))
df = pd.melt(df, id_vars=['user_id', 'date', 'device', 'sex'], value_name='page').drop(['variable'],axis=1)
print(df)

   user_id     page_x page_y page_x       page_y        date   device     sex
0   313593  home_page    NaN    NaN          NaN  2015-02-26  Desktop  Female
1   468315  home_page    NaN    NaN          NaN  2015-02-21  Desktop    Male
2   264005  home_page    NaN    NaN          NaN  2015-03-25  Desktop  Female
3   290784  home_page    NaN    NaN          NaN  2015-03-14  Desktop    Male
4   639104  home_page    NaN    NaN  search_page  2015-01-03  Desktop  Female
5   262889  home_page    NaN    NaN          NaN  2015-01-10   Mobile    Male
6   708793  home_page    NaN    NaN          NaN  2015-04-24   Mobile  Female
7   943143  home_page    NaN    NaN  search_page  2015-04-10  Desktop    Male
8   417894  home_page    NaN    NaN          NaN  2015-04-02   Mobile  Female
9   729374  home_page    NaN    NaN  search_page  2015-04-12  Desktop    Male
        user_id        date   device     sex         page
0        313593  2015-02-26  Desktop  Female    home_page
1        468315  2015-02-2

In [65]:
GROUP_KEY = 'user_id'
TIME_FIELD = 'date'
EVENT_FIELD = 'page'
FUNNEL_STEPS = ['home_page', 'search_page', 'payment_page', 'payment_confirmation_page']


def funnelize(group, funnel_steps):
    curr = 0
    funnel_cts = [0 for s in funnel_steps]

    for i, row in group.iterrows():
        evt = row[EVENT_FIELD]
        if evt in funnel_steps:
            idx = funnel_steps.index(evt)
            if idx <= curr + 1:
                funnel_cts[idx] += 1
                curr = idx
        
    return funnel_cts


# sort event dataset by time
df.sort_values(by=GROUP_KEY, ascending=True, inplace=True)

# step through events by group_key (i.e., browser, visit), 
# and increment event counts by funnel step if prior funnel steps occurred
funnel_cts = df.groupby(GROUP_KEY).apply(funnelize, FUNNEL_STEPS)

# format results as a dataframe
funnel = pd.DataFrame(list(funnel_cts.values), index=funnel_cts.index, columns=FUNNEL_STEPS)

funnel.head()

,home_page,search_page,payment_page,payment_confirmation_page
user_id,,,,
17,1,1,0,0
28,1,0,0,0
37,1,1,0,0
38,1,1,0,0
55,1,0,0,0


In [66]:
# funnel summary (aggregate)

print (funnel[FUNNEL_STEPS].sum())

home_page                    90400
search_page                  45200
payment_page                  2082
payment_confirmation_page       46
dtype: int64


In [88]:
# funnel summary, deduped across group key 
# (i.e., event occurrences become 1/0 indicators instead of counts)

funnel_norm = funnel.copy()
funnel_norm[funnel_norm != 0] = 1
funnel_analysis = funnel_norm[FUNNEL_STEPS].sum()
print (funnel_analysis)

home_page                    90400
search_page                  45200
payment_page                  2082
payment_confirmation_page       46
dtype: int64


In [100]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from __future__ import division

colors = ['rgb(32,155,160)', 'rgb(253,93,124)', 'rgb(28,119,139)', 'rgb(182,231,235)']
phases = list(funnel_analysis.index)
values = list(funnel_analysis)

In [96]:
n_phase = len(phases)
 
# the fixed width for the plot
plot_width = 400
 
# height of a section and difference between sections 
section_h = 100
section_d = 10
 
# multiply factor to calculate the width of other sections
unit_width = plot_width / max(values)
 
# width for all the sections (phases)
phase_w = [int(value * unit_width) for value in values]

In [97]:
height = section_h * n_phase + section_d * (n_phase-1)
 
shapes = []
 
label_y = []
 
for i in range(n_phase):
        if (i == n_phase-1):
                points = [phase_w[i]/2, height, phase_w[i]/2, height - section_h]
        else:
                points = [phase_w[i]/2, height, phase_w[i+1]/2, height - section_h]
 
        path = 'M {0} {1} L {2} {3} L -{2} {3} L -{0} {1} Z'.format(*points)
 
        shape = {
                'type': 'path',
                'path': path,
                'fillcolor': colors[i],
                'line': {
                    'width': 1,
                    'color': colors[i]
                }
        }
        shapes.append(shape)
        
        # Y-axis location for this section's details (phase name and value)
        label_y.append(height - (section_h / 2))
 
        height = height - (section_h + section_d)

In [98]:
# For phase names
label_trace = go.Scatter(
    x=[-350]*n_phase,
    y=label_y,
    mode='text',
    text=phases,
    textfont=dict(
        color='rgb(200,200,200)',
        size=15
    )
)
 
# For phase values
value_trace = go.Scatter(
    x=[350]*n_phase,
    y=label_y,
    mode='text',
    text=values,
    textfont=dict(
        color='rgb(200,200,200)',
        size=15
    )
)

In [101]:
data = [label_trace, value_trace]
 
layout = go.Layout(
    title='Funnel Chart',
    shapes=shapes,
    height=560,
    width=800,
    showlegend=False,
    paper_bgcolor='rgba(44,58,71,1)',
    plot_bgcolor='rgba(44,58,71,1)',
    xaxis=dict(
        showticklabels=False,
        zeroline=False,
    ),
    yaxis=dict(
        showticklabels=False,
        zeroline=False
    )
)
 
fig = go.Figure(data=data, layout=layout)
iplot(fig)